In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
loader = PyPDFLoader('./atp7-100-1_defense.pdf')
pages = loader.load_and_split() # pdf 페이지 단위로 쪼깨서 읽어 들임
print(len(pages))

17


In [5]:
pages[1]

Document(metadata={'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-05-23T07:42:04+00:00', 'source': './atp7-100-1_defense.pdf', 'total_pages': 14, 'page': 1, 'page_label': '2'}, page_content="Chapter 6 \n6-2 ATP 7-100.1 \x15\x1c\x03)HEUXDU\\\x03\x15\x13\x15\x17 \nACHIEVING STABILITY IN THE DEFENSE \n6-4. To achieve stability in the defense, SV ground units must be able to survive impacts of nuclear weapons \nand maintain a low exposure to aggressor precision weap ons. To accomplish this, units must be dispersed,\ncovered, concealed, and use resources to mask thei r positions from radar and thermal imaging devices. To\nmaintain stability, SV ground units mu st also have the means to protec t against precision munitions using\nelectromagnetic warfare. Stability measures also include—\nz Antitank capability to deter attacks by tanks and armored vehicles.\nz Ability to defend against tactical and operational air strikes.\nz Ability to resist airborne and ai

In [6]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=0,
    encoding_name="cl100k_base",
)
texts = text_splitter.split_documents(pages)

In [7]:
len(texts)

17

In [8]:
texts[0].page_content

'\x15\x1c\x03)HEUXDU\\\x03\x15\x13\x15\x17 ATP 7-100.1 6-1\nChapter 6 \nDefense \nOver the course of the last two decades, the Ground Forces of the Russian Federation \n\x0bɋɭɯɨɩɭɬɧɵɟ\x03ɜɨɣɫɤɚ\x03Ɋɨɫɫɢɣɫɤɨɣ\x03Ɏɟɞɟɪɚɰɢɢ\x03– sukhoputnyye voyska RF - SV) have \nemphasized defensive tactics. And, while SV units train in both positional and \nmaneuver defense, recent SV doctrinal papers have emphasized the value of the \nmaneuver defense over the positional defense. In all cases however, the SV considers \nthe tactical counterattack as the ultimate method for destroying an aggressor on the \nbattlefield. This chapter explains the theory and rational behind the SV’s defensive \nobjectives, as well as, how the SV design, arrange, and execute tactical defensive \nactions. Additionally, the tactical defensive requirements and characteristics for units \nare described in detail.  \nPURPOSE OF THE DEFENSE \n6-1. The SV is not the ground force of World War II or the Cold War, which involved mas

In [9]:
texts[1].page_content

"Chapter 6 \n6-2 ATP 7-100.1 \x15\x1c\x03)HEUXDU\\\x03\x15\x13\x15\x17 \nACHIEVING STABILITY IN THE DEFENSE \n6-4. To achieve stability in the defense, SV ground units must be able to survive impacts of nuclear weapons \nand maintain a low exposure to aggressor precision weap ons. To accomplish this, units must be dispersed,\ncovered, concealed, and use resources to mask thei r positions from radar and thermal imaging devices. To\nmaintain stability, SV ground units mu st also have the means to protec t against precision munitions using\nelectromagnetic warfare. Stability measures also include—\nz Antitank capability to deter attacks by tanks and armored vehicles.\nz Ability to defend against tactical and operational air strikes.\nz Ability to resist airborne and air assault attacks and aggressor reconnaissance efforts as well as the\ncapability to deter attempted sabotage.\nELEMENTS OF AN ACTIVE DEFENSE \n6-5. An active defense must be capable of placing the aggressor under planned an

In [10]:
texts[2].page_content

"Defense \n\x15\x1c\x03)HEUXDU\\\x03\x15\x13\x15\x17 ATP 7-100.1 6-3\naggressor counterattacks, hold key terrain, or to protect exposed flanks before shifting to offensive operations. \nSV manuals stipulate that a brigade's main defensive zone in a positional defense will be up to 15 km wide \nand up to 20 km deep. As always, though, the width and depth will be situationally dependent. \nMANEUVER DEFENSE \n6-10. When circumstances arise in which the aggressor has substantial advantage over SV forces, and when\nit is possible to trade maneuver space for time, the SV will use a maneuver defense. Maneuver defense is\ndesigned to win time, allow for the reorganizing of forces, and at the intended time, strike a critical blow on\nthe attacking aggressor. In short, the maneuver defense is achieved by echeloned, sequential defensive actions \nplanned, and culminating in a decisive blow against the aggressor forces. A brigade's main defensive front in\na maneuver defense can be up to 20 km wid

In [11]:
model = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en-v1.5')

C:\Users\sooyeonkim\AppData\Local\Temp\ipykernel_21804\260948898.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name='BAAI/bge-large-en-v1.5')


In [12]:
chroma_db = Chroma.from_documents(texts, model)

In [13]:
chroma_db._collection.count()

17

In [14]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
import os

In [15]:
os.environ['OPENAI_API_KEY'] = ''

In [32]:
template = '''너는 군사 전문가이고, 검색 결과를 바탕으로 똑바로 답해야 해.
만약 모르겠다면, 솔직하게 모른다고 해. 그리고 이모지를 넣어서 친근하게 답해봐!!! 
{context}

question: {question}
answer:
'''

prompt = PromptTemplate.from_template(template)

In [17]:
llm = ChatOpenAI(model_name='gpt-4o')

In [18]:
result = llm.invoke('hi! who are you?')
print(result.content)

Hello! I'm an AI language model created by OpenAI. I'm here to help answer questions, provide information, and assist with a wide range of topics. How can I assist you today?


In [20]:
retriever = chroma_db.as_retriever(search_kwargs={'k': 2})

In [33]:
# 랭체인 핵심 ! 

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={'prompt': prompt},
    return_source_documents=True,
)

In [23]:
question = "What is the main topic of the document?"
chatbot_response = qa_chain.invoke(question)

In [24]:
chatbot_response

{'query': 'What is the main topic of the document?',
 'result': 'The main topic of the document is the defensive tactics and strategies of the Ground Forces of the Russian Federation (SV). It elaborates on the theory, goals, and requirements behind their defensive objectives, and details the considerations and responsibilities of commanders at various levels in arranging and executing tactical defensive actions.',
 'source_documents': [Document(metadata={'producer': 'iLovePDF', 'total_pages': 14, 'moddate': '2025-05-23T07:42:04+00:00', 'source': './atp7-100-1_defense.pdf', 'page_label': '1', 'creationdate': '', 'page': 0, 'creator': 'PyPDF'}, page_content='\x15\x1c\x03)HEUXDU\\\x03\x15\x13\x15\x17 ATP 7-100.1 6-1\nChapter 6 \nDefense \nOver the course of the last two decades, the Ground Forces of the Russian Federation \n\x0bɋɭɯɨɩɭɬɧɵɟ\x03ɜɨɣɫɤɚ\x03Ɋɨɫɫɢɣɫɤɨɣ\x03Ɏɟɞɟɪɚɰɢɢ\x03– sukhoputnyye voyska RF - SV) have \nemphasized defensive tactics. And, while SV units train in both positional

In [25]:
print(chatbot_response['result'])

The main topic of the document is the defensive tactics and strategies of the Ground Forces of the Russian Federation (SV). It elaborates on the theory, goals, and requirements behind their defensive objectives, and details the considerations and responsibilities of commanders at various levels in arranging and executing tactical defensive actions.


In [37]:
question = "What information do you need when deploying defensive tactics?"
chatbot_response = qa_chain.invoke(question)
print(chatbot_response['result'])

When deploying defensive tactics, several key pieces of information are needed to ensure effective planning and execution. These include:

1. **Understanding the Terrain**: Knowledge of the geographical features that can be used for defensive advantages, such as natural obstacles, vantage points, and areas for concealment.

2. **Enemy Capabilities and Movement**: Information about the aggressor’s strength, tactics, movement, and possible attack trajectories.

3. **Friendly Force Composition**: Details of the units available, including their strengths and specialties, which will influence tactical decisions.

4. **Logistics and Supply Lines**: Assurance that resources such as ammunition, food, water, and medical supplies are available and can be sustained during operations.

5. **Communication Systems**: Reliable and secure lines of communication for effective command and control.

6. **Weather Conditions**: Weather can significantly impact operations, affecting both movement and visibi

In [34]:
question = "방어 전술을 펼칠때, 필요한 정보가 뭐야?"
chatbot_response = qa_chain.invoke(question)
print(chatbot_response['result'])

방어 전술을 펼칠 때 필요한 정보는 여러 가지 요소를 포함합니다. 😊 이 정보들은 방어를 효과적으로 계획하고 실행하기 위해 중요합니다. 여기 몇 가지 주요 고려사항을 정리해 봅니다:

1. **증원 및 주요 강점**: 방어 위치를 강화하고 집중해야 할 방향을 결정합니다.
2. **공격 반격 준비**: 적의 침입에 대한 대비 계획을 수립합니다.
3. **화력 배치**: 사격 범위와 집중 사격 구역을 설정합니다.
4. **옆구리 및 간격 보호**: 측면 보호와 갭을 방어하는데 필요한 병력과 자원을 결정합니다.
5. **전술적 반격**: 필요 시 반격을 준비합니다.
6. **장비 위치**: 탱크 및 기동 보병 부대의 사격선과 배치를 고려합니다.
7. **지원 화력**: 포병과 박격포 지원 계획을 수립합니다.
8. **대전차 및 대공 방어**: 적의 기갑 및 공중 공격에 대한 대비책을 마련합니다.
9. **경고 시스템**: 적 공중 공격을 탐지하기 위한 정찰 및 조기 경보 시스템을 배치합니다.

이런 요소들은 방어 전략에서 안정성과 활동성을 유지하기 위해 필수적입니다. 🛡️🗺️


In [35]:
question = "문서에 등장하는 장비 이름을 알려줘"
chatbot_response = qa_chain.invoke(question)
print(chatbot_response['result'])

문서에 등장하는 장비 이름은 다음과 같습니다:

- 탱크 (tanks)
- BMP (보병전투차량)
- BTR (장갑차량)
- 대전차 미사일 (antitank missiles)
- 유탄 발사기 (grenade launchers)
- 소화기 (small arms)
- 화염 및 소이 무기 (flame and incendiary weapons)
- 항공 자원 (aviation resources)

이 장비들은 방어 작전에서 효과적인 화력 파괴 시스템을 구축하는 데 사용됩니다. 🚜🔫✈️


In [36]:
question = "문서에 등장하는 장비 이름을 영어로 알려줘"
chatbot_response = qa_chain.invoke(question)
print(chatbot_response['result'])

🤔 문서에서 언급된 장비는 다음과 같습니다:

- Artillery
- Tanks
- BMPs (Boyevaya Mashina Pekhoty, 보병 전투 차량)
- BTRs (Bronetransportyor, 장갑 수송차량)
- Antitank missiles
- Grenade launchers
- Small arms
- Flame and incendiary weapons (thermobaric)
- Aviation resources (Air strikes and air defense weapons)

이 중에서 정해진 영문 약어와 번역이 함께 있는 장비들이 포함되어 있습니다. 😊
